<a href="https://colab.research.google.com/github/nikitakandpal/ML-projects/blob/main/Song_popularity_predictive_Model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
data = pd.read_csv("song_data.csv")

In [3]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 18835 entries, 0 to 18834
Data columns (total 15 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   song_name         18835 non-null  object 
 1   song_popularity   18835 non-null  int64  
 2   song_duration_ms  18835 non-null  int64  
 3   acousticness      18835 non-null  float64
 4   danceability      18835 non-null  float64
 5   energy            18835 non-null  float64
 6   instrumentalness  18835 non-null  float64
 7   key               18835 non-null  int64  
 8   liveness          18835 non-null  float64
 9   loudness          18835 non-null  float64
 10  audio_mode        18835 non-null  int64  
 11  speechiness       18835 non-null  float64
 12  tempo             18835 non-null  float64
 13  time_signature    18835 non-null  int64  
 14  audio_valence     18835 non-null  float64
dtypes: float64(9), int64(5), object(1)
memory usage: 2.2+ MB


In [4]:
for col in data.columns:
  print(col, ':',len(data[col].unique()), 'labels')

song_name : 13070 labels
song_popularity : 101 labels
song_duration_ms : 11771 labels
acousticness : 3209 labels
danceability : 849 labels
energy : 1132 labels
instrumentalness : 3925 labels
key : 12 labels
liveness : 1425 labels
loudness : 8416 labels
audio_mode : 2 labels
speechiness : 1224 labels
tempo : 12112 labels
time_signature : 5 labels
audio_valence : 1246 labels
